Experimenting with estimating the gender of mailing list participants.

In [1]:
%matplotlib inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Import the BigBang modules as needed. These should be in your Python environment if you've installed BigBang correctly.

In [2]:
import bigbang.mailman as mailman
import bigbang.graph as graph
import bigbang.process as process
from bigbang.parse import get_date
from bigbang.archive import Archive
reload(process)

import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np
import math
import pytz
import pickle
import os

from bigbang import parse
from gender_detector import GenderDetector

pd.options.display.mpl_style = 'default' # pandas has a set of preferred graph formatting options

<module 'bigbang.process' from '/Users/nick/code/mailing-list-analysis/bigbang/bigbang/process.pyc'>

Now let's load the data for analysis.

In [69]:
urls = ["http://www.ietf.org/mail-archive/text/ietf-privacy/",
        "http://lists.w3.org/Archives/Public/public-privacy/"]
mlists = [(url, mailman.open_list_archives(url,"../archives")) for url in urls]
activities = [Archive.get_activity(Archive(ml)) for ml in mlists]

Opening 36 archive files
Opening 17 archive files


TypeError: 'NoneType' object has no attribute '__getitem__'

For each of our lists, we'll clean up the names, find the first name if there is one, and guess its gender. Pandas groups the data together for comparison.

In [68]:
detector = GenderDetector('us')

def guess_gender(name):
    if not name:
        return 'name unknown'
    try:
        return detector.guess(name)
    except:
        return 'error'

series = []  
for (url, ml) in mlists:
    ml['Name'] = ml['From'].map(lambda x: parse.clean_from(x))
    ml['First Name'] = ml['Name'].map(lambda x: parse.guess_first_name(x))
    ml['Guessed Gender'] = ml['First Name'].map(guess_gender)
    counts = ml.groupby('Guessed Gender')['From'].count()
    #ml.groupby('Guessed Gender').size()
    counts.name=url
    series.append(counts)

pd.DataFrame(series)

,error,female,male,name unknown,unknown
http://www.ietf.org/mail-archive/text/ietf-privacy/,NaN,32.0,195.0,109.0,166.0
http://lists.w3.org/Archives/Public/public-privacy/,1.0,211.0,826.0,200.0,179.0
